# Import Reference Data from CSV Files


In [ ]:
import os
import logging


In [ ]:
# Parameters

keyword_to_concept_csv_file= "../Data/TechINT/reference/keywords.csv"
concept_to_concept_csv_file= "../Data/TechINT/reference/concepts.csv"

# Logging level ranges are (from least to most verbose): ERROR, WARN, INFO, DEBUG
logging_level = logging.INFO

# level of unnecessary output
verbose = True

In [ ]:
logging.getLogger("OpenTLDR").setLevel(logging_level)

from opentldr import KnowledgeGraph
from opentldr.Domain import ReferenceNode, ReferenceEdge

kg=KnowledgeGraph()

## Example Loading of Keyword Reference Data from a CSV file

- Keywords are loaded as ReferenceNodes where the type is KEYWORD
- Concepts are loaded as ReferenceNodes where the type is CONCEPT
- Relationships link the KEYWORD to the CONCEPT

### Import Entities for Reference Data from a CSV file

CVS Format for Keyword and Concept:

| Keyword | Concept | Description |
| :---: | :--- | :--- |
| bacon | breakfast foods | People like bacon for breakfast. |
| eggs | breakfast foods | People eat eggs for breakfast. |

In [ ]:

unique_keywords = {}
unique_concepts = {}

if keyword_to_concept_csv_file is not None:
    with open(keyword_to_concept_csv_file) as file:
        lines = file.readlines()
        for line in lines[1:]:  # Note: skips the header line in the CSV file 
            # parse out some fields from the csv record
            part = line.split(",")
            keyword = part[0].strip('\"').strip()
            concept = part[1].strip('\"').strip()
            desc = part[2].strip('\"').strip()

            # You can call the KnowledgeGraph API for most actions
            concept_node=None
            if concept.lower() in unique_concepts:
                concept_node = unique_concepts[concept.lower()]
            else:
                concept_node = kg.add_reference_node(text=concept,type="CONCEPT")
                unique_concepts[concept.lower()] = concept_node
            
            keyword_node=None
            if keyword.lower() in unique_keywords:
                keyword_node = unique_keywords[keyword.lower()]
            else:
                keyword_node= kg.add_reference_node(text=keyword,type="KEYWORD")
                unique_keywords[keyword.lower()] = keyword_node

            edge = kg.add_reference_edge(from_node=keyword_node,to_node=concept_node, type="KeywordToConcept", text=desc)

            print("Added ({uid}):\t{text} of type {type}".format(uid=keyword_node.uid, text=keyword_node.text, type=keyword_node.type))

### Import Relationships between Concepts for Reference Data from a CSV file

CVS Format for Relationships:

| Concept From | Concept To | Relationship Text |
| :---: | :---: | :---: |
| breakfast foods | foods | Breakfast foods relate to the category of foods in general. |


In [ ]:
if concept_to_concept_csv_file is not None:
    with open(concept_to_concept_csv_file) as file:
        lines = file.readlines()
        for line in lines[1:]:  # Note: skips the header line in the CSV file 
            # parse out some fields from the csv record
            part = line.split(",")
            from_concept = part[0].strip('\"').strip()
            to_concept = part[1].strip('\"').strip()
            desc = part[2].strip('\"').strip()

            # You can call the KnowledgeGraph API for most actions

            from_concept_node=None
            if from_concept.lower() in unique_concepts:
                from_concept_node = unique_concepts[from_concept.lower()]
            else:
                from_concept_node = kg.add_reference_node(text=from_concept,type="CONCEPT")
                unique_concepts[from_concept.lower()] = from_concept_node
                print("Added Concept ({uid}):\t{text}".format(uid=from_concept_node.uid, text=from_concept_node.text))

            to_concept_node=None
            if to_concept.lower() in unique_concepts:
                to_concept_node = unique_concepts[to_concept.lower()]
            else:
                to_concept_node = kg.add_reference_node(text=to_concept,type="CONCEPT")
                unique_concepts[to_concept.lower()] = to_concept_node
                print("Added Concept ({uid}):\t{text}".format(uid=to_concept_node.uid, text=to_concept_node.text))


            edge = kg.add_reference_edge(from_node=from_concept_node,to_node=to_concept_node,type="ConceptToConcept",text=desc)

            print("Added Relationship ({uid}):\t{text}".format(uid=edge.uid, text=edge.text))


# Close down the remote connections to the database

In [ ]:
kg.close()